In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
train_df = pd.read_csv(f"D:/teclab/Machine learning/train.csv")
test_df = pd.read_csv(f"D:/teclab/Machine learning/test.csv")

In [ ]:
train_df.dtypes, test_df.dtypes

(PassengerId      int64
 Survived         int64
 Pclass           int64
 Name            object
 Sex             object
 Age            float64
 SibSp            int64
 Parch            int64
 Ticket          object
 Fare           float64
 Cabin           object
 Embarked        object
 dtype: object,
 PassengerId      int64
 Pclass           int64
 Name            object
 Sex             object
 Age            float64
 SibSp            int64
 Parch            int64
 Ticket          object
 Fare           float64
 Cabin           object
 Embarked        object
 dtype: object)

In [ ]:
train_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
train_df.isnull().sum(), test_df.isnull().sum()

(PassengerId      0
 Survived         0
 Pclass           0
 Name             0
 Sex              0
 Age            177
 SibSp            0
 Parch            0
 Ticket           0
 Fare             0
 Cabin          687
 Embarked         2
 dtype: int64,
 PassengerId      0
 Pclass           0
 Name             0
 Sex              0
 Age             86
 SibSp            0
 Parch            0
 Ticket           0
 Fare             1
 Cabin          327
 Embarked         0
 dtype: int64)

In [ ]:
# Rellenar Age con la mediana del conjunto de entrenamiento
agea_median = train_df['Age'].median()
# Corregido: Asignar el resultado de fillna de vuelta a la columna sin inplace=True
train_df['Age'] = train_df['Age'].fillna(agea_median)
# Rellenar ahora Age con la mediana del conjunto de prueba
ageb_median = test_df['Age'].median()
# Corregido: Asignar el resultado de fillna de vuelta a la columna sin inplace=True
test_df['Age'] = test_df['Age'].fillna(ageb_median)

In [ ]:
# Rellenar Fare tanto en train como en test con la mediana de su conjunto respectibamente.
farea_median = train_df['Fare'].median()
train_df['Fare'] = train_df['Fare'].fillna(farea_median)
fareb_median = test_df['Fare'].median()
test_df['Fare'] = test_df['Fare'].fillna(fareb_median)

In [ ]:
# Rellenar Cabin con 'Unknown' (Desconocido)
train_df['Cabin'] = train_df['Cabin'].fillna('Unknown')
test_df['Cabin'] = test_df['Cabin'].fillna('Unknown')

# Rellenar Embarked en train con mode() del conjunto de entrenamiento
embarked_mode = train_df['Embarked'].mode()[0]
train_df['Embarked'] = train_df['Embarked'].fillna(embarked_mode)

In [ ]:
# Verificar que no queden valores faltantes
print("\nValores faltantes después de la imputación (entrenamiento):")
print(train_df.isnull().sum())
print("\nValores faltantes después de la imputación (prueba):")
print(test_df.isnull().sum())


Valores faltantes después de la imputación (entrenamiento):
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

Valores faltantes después de la imputación (prueba):
PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64


In [ ]:
# Convertir variables categóricas a numéricas
train_df['Sex'] = train_df['Sex'].map({'male': 0, 'female': 1})
test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1})

In [ ]:
# Convertir Embarked a variables dummy
train_df = pd.get_dummies (train_df, columns=['Embarked'], drop_first=True)
test_df = pd.get_dummies (test_df, columns=['Embarked'], drop_first=True)

KeyError: "None of [Index(['Embarked'], dtype='object')] are in the [columns]"

In [ ]:
# Seleccionar características para el modelo
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_Q', 'Embarked_S']
X = train_df[features]
y = train_df['Survived']

In [ ]:
# Dividir datos de entrenamiento para validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Escalar características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(test_df[features])

KeyError: "['Embarked_Q', 'Embarked_S'] not in index"

In [ ]:
# --- Se agregó una verificación antes de escalar los datos de prueba ---
print("\nVerificando NaNs en test_df antes de seleccionar características y escalar:")
print(test_df[features].isnull().sum())


Verificando NaNs en test_df antes de seleccionar características y escalar:


KeyError: "['Embarked_Q', 'Embarked_S'] not in index"

In [ ]:
# Asegurarse de que test_df tenga las mismas columnas que X antes de escalar
# Este paso puede parecer redundante, pero previene posibles discrepancias de columnas
X_test = test_df[features]
print("\nVerificando NaNs en X_test antes de escalar:")
print(X_test.isnull().sum())

KeyError: "['Embarked_Q', 'Embarked_S'] not in index"

In [ ]:
# Escalar los datos de prueba
X_test_scaled = scaler.transform(X_test)

print("\nVerificando NaNs en X_test_scaled después de escalar:")
print(pd.DataFrame(X_test_scaled, columns=features).isnull().sum())
print(" --- Fin de la verificación agregada --- ")


Verificando NaNs en X_test_scaled después de escalar:
Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked_Q    0
Embarked_S    0
dtype: int64
 --- Fin de la verificación agregada --- 


In [ ]:
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [ ]:
#Validacion cruzada X5
scores = cross_val_score(model, X_train_scaled, y_train, cv=5)
print("Precision de la validacion cruzada:", scores.mean())

Precision de la validacion cruzada: 0.7892544075642667


In [ ]:
# Predecir en el conjunto de validación
y_val_pred = model.predict(X_val_scaled)
predictions = model.predict(X_val_scaled)
print(predictions[0:5])

[0 0 0 1 1]


In [ ]:
model.score(X_val_scaled,y_val_pred)

1.0

In [ ]:
# Calcular la precisión
accuracy = accuracy_score(y_val, y_val_pred)
print(f"\nPrecisión del modelo en el conjunto de validación: {accuracy:.4f}")


Precisión del modelo en el conjunto de validación: 0.8101


In [ ]:
# Predecir en el conjunto de prueba (para presentación o evaluación adicional)
test_predictions = model.predict(X_test_scaled)
#Ver resultados de la prediccion.
print(test_predictions[0:20])

[0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1]


In [ ]:
# Guardar predicciones en un archivo CSV
output = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived': test_predictions})
output.to_csv('predicciones_titanic.csv', index=False)
print("\nPredicciones del conjunto de prueba guardadas en 'predicciones_titanic.csv'")


Predicciones del conjunto de prueba guardadas en 'predicciones_titanic.csv'


In [ ]:
output.shape


(418, 2)

In [ ]:
output

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
